<a href="https://colab.research.google.com/github/AMD2019/Python-for-Data-Science/blob/master/SDGs_revised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: File :  SDGs_revised file sheet "SDGs" path "/content/SDGs_revised.xlsx" has following headers [Ind_Code      Targets and Indicators  Indicator       2015    2019    2022    2025    2030    "Sources of Data" "Level of Disaggregation" Frequency "Responsible Agency (Reporting)" "Responsible Agency (Data)"]
# Ind_Code code starts with "C" (7 characters), "N" (9 characters) or word "Target"
# There are some rows in Ind_Code split into multiple rows. It is because they erroneously include values that should have been in "Targets and Indicators" or data belonging to other fields. The split rows immediately follows the main row. If the row starts with "C" (7 characters), "N" (9 characters) or word "Target" you can identify it as beginning of new row.
# You will have to apply machine learning technique to clean the data, save it as "SDGs_clean" excel. There must be more than 400 indicators. While cleaning iterate until you reach more than 400 indicators
# After saving the file apply text matching technique to produce the matching score. Text matching or conceptual matching. Save the file as "Similarlity_Score_Revised". Include
# "Ind_Code" "Targets and Indicators"   "Indicator" as columns.

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def clean_sdgs_data(df):
    cleaned_data = []
    current_row = None

    for index, row in df.iterrows():
        ind_code = row.get('Ind_Code') # Use .get() to avoid KeyError

        if pd.notna(ind_code) and (
            (isinstance(ind_code, str) and ind_code.startswith('C') and len(ind_code) == 7) or
            (isinstance(ind_code, str) and ind_code.startswith('N') and len(ind_code) == 9) or
            (isinstance(ind_code, str) and ind_code == 'Target')
        ):
            # Start of a new indicator or target row
            if current_row is not None:
                cleaned_data.append(current_row)
            current_row = row.copy()
        elif current_row is not None:
            # Continuation of the previous row, merge relevant columns
            for col in ['Targets and Indicators', 'Indicator', 'Sources of_x000D_Data', 'Level of_x000D_Disaggregation', 'Frequency', 'Responsible_x000D_Agency_x000D_(Reporting)', '"Responsible_x000D_Agency_x000D_(Data)"']:
                 if col in row and pd.notna(row[col]): # Check if column exists
                    if pd.notna(current_row.get(col)): # Use .get()
                         current_row[col] = str(current_row[col]) + " " + str(row[col])
                    else:
                         current_row[col] = row[col]
            # Attempt to merge numerical columns where appropriate (assuming non-empty rows imply data points)
            for year_col in ['2015', '2019', '2022', '2025', '2030']:
                 if year_col in row and pd.notna(row[year_col]): # Check if column exists
                      # Simple concatenation for now; a more sophisticated approach might average or choose based on rules
                      if pd.notna(current_row.get(year_col)): # Use .get()
                           current_row[year_col] = str(current_row[year_col]) + "," + str(row[year_col])
                      else:
                           current_row[year_col] = row[year_col]


    # Add the last collected row
    if current_row is not None:
        cleaned_data.append(current_row)

    cleaned_df = pd.DataFrame(cleaned_data)
    return cleaned_df

# Load the original data
df = None  # Initialize df to None
try:
    df = pd.read_excel("/content/SDGs_revised_.xlsx", sheet_name="SDGs")
except FileNotFoundError:
    print("Error: The file '/content/SDGs_revised.xlsx' was not found.")


# Proceed only if the dataframe was loaded successfully
if df is not None:
    # Initialize a flag to continue cleaning until we reach enough indicators
    cleaned_df = None
    iteration = 0
    min_indicators = 400

    while cleaned_df is None or (cleaned_df['Ind_Code'].astype(str).str.startswith(('C', 'N')).sum() < min_indicators and iteration < 10): # Add an iteration limit to prevent infinite loops
        print(f"Cleaning iteration: {iteration}")
        if iteration == 0:
          cleaned_df = clean_sdgs_data(df.copy())
        else:
           # If we are in subsequent iterations, re-run cleaning on the potentially partially cleaned data
           cleaned_df = clean_sdgs_data(cleaned_df.copy())
        iteration += 1
        # Ensure 'Ind_Code' column exists before attempting to use it
        if 'Ind_Code' in cleaned_df.columns:
            print(f"Number of indicators after cleaning: {cleaned_df['Ind_Code'].astype(str).str.startswith(('C', 'N'), na=False).sum()}")
        else:
            print("Ind_Code column not found in the cleaned DataFrame.")
            break # Exit loop if 'Ind_Code' is not present


    # Filter for rows that are indicators (starting with C or N)
    # Ensure 'Ind_Code' column exists before attempting to use it
    if 'Ind_Code' in cleaned_df.columns:
        indicator_df = cleaned_df[cleaned_df['Ind_Code'].astype(str).str.startswith(('C', 'N'), na=False)].copy()

        # Save the cleaned data
        cleaned_output_path = "/content/SDGs_clean.xlsx"
        indicator_df.to_excel(cleaned_output_path, index=False)
        print(f"Cleaned data saved to: {cleaned_output_path}")


        # --- Apply text matching technique ---

        # Combine 'Targets and Indicators' and 'Indicator' for similarity calculation
        indicator_df['combined_text'] = indicator_df['Targets and Indicators'].astype(str).fillna('') + ' ' + indicator_df['Indicator'].astype(str).fillna('')

        # Initialize TF-IDF Vectorizer
        tfidf_vectorizer = TfidfVectorizer()

        # Fit and transform the combined text data
        tfidf_matrix = tfidf_vectorizer.fit_transform(indicator_df['combined_text'])

        # Calculate cosine similarity between indicators
        cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

        # Create a DataFrame for similarity scores
        # We will store pairs of indicators and their similarity score
        similarity_data = []
        for i in range(len(indicator_df)):
            for j in range(i + 1, len(indicator_df)): # Avoid self-comparison and duplicate pairs
                ind_code_i = indicator_df.iloc[i]['Ind_Code']
                ind_code_j = indicator_df.iloc[j]['Ind_Code']
                target_indicator_i = indicator_df.iloc[i]['Targets and Indicators']
                target_indicator_j = indicator_df.iloc[j]['Targets and Indicators']
                indicator_i = indicator_df.iloc[i]['Indicator']
                indicator_j = indicator_df.iloc[j]['Indicator']
                score = cosine_sim[i, j]

                similarity_data.append({
                    "Ind_Code_1": ind_code_i,
                    "Targets and Indicators_1": target_indicator_i,
                    "Indicator_1": indicator_i,
                    "Ind_Code_2": ind_code_j,
                    "Targets and Indicators_2": target_indicator_j,
                    "Indicator_2": indicator_j,
                    "Similarity_Score": score
                })

        similarity_df = pd.DataFrame(similarity_data)

        # Sort by similarity score in descending order to see the most similar pairs first
        similarity_df = similarity_df.sort_values(by="Similarity_Score", ascending=False)


        # Save the similarity score results
        similarity_output_path = "/content/Similarlity_Score_Revised.xlsx"
        # Include the requested columns: "Ind_Code", "Targets and Indicators", "Indicator" for each pair
        similarity_df = similarity_df[['Ind_Code_1', 'Targets and Indicators_1', 'Indicator_1', 'Ind_Code_2', 'Targets and Indicators_2', 'Indicator_2', 'Similarity_Score']]
        similarity_df.to_excel(similarity_output_path, index=False)
        print(f"Similarity scores saved to: {similarity_output_path}")
    else:
        print("Could not proceed with similarity calculation as 'Ind_Code' column was not found in the cleaned DataFrame.")
else:
    print("DataFrame was not loaded due to file error. Skipping further processing.")

Cleaning iteration: 0
Number of indicators after cleaning: 367
Cleaning iteration: 1
Number of indicators after cleaning: 367
Cleaning iteration: 2
Number of indicators after cleaning: 367
Cleaning iteration: 3
Number of indicators after cleaning: 367
Cleaning iteration: 4
Number of indicators after cleaning: 367
Cleaning iteration: 5
Number of indicators after cleaning: 367
Cleaning iteration: 6
Number of indicators after cleaning: 367
Cleaning iteration: 7
Number of indicators after cleaning: 367
Cleaning iteration: 8
Number of indicators after cleaning: 367
Cleaning iteration: 9
Number of indicators after cleaning: 367
Cleaned data saved to: /content/SDGs_clean.xlsx
Similarity scores saved to: /content/Similarlity_Score_Revised.xlsx
